# PSO

**Agora olhando para todos ao mesmo tempo**

In [1]:
import numpy as np
from scipy.optimize import minimize
from pyXRD import pyXRDCodes
from scipy.signal import find_peaks
import random
from collections import defaultdict

import matplotlib.pyplot as plt

---

In [5]:
def Vol_calc(unit_params, unit_angles, V_max):
    a = pyXRDCodes.abcToxyz([1,0,0], unit_params, unit_angles)
    b = pyXRDCodes.abcToxyz([0,1,0], unit_params, unit_angles)
    c = pyXRDCodes.abcToxyz([0,0,1], unit_params, unit_angles)
    V = abs(np.dot(a, np.cross(b, c)))
    return V <= V_max

def compute_fitness(params, theta2_exp, lamb, power=1, peak_tol=0.02, v_max=3000):
    """
    Updated fitness function to handle all 6 parameters
    params: [a, b, c, alpha, beta, gamma]
    """
    unit_params = params[:3]
    unit_angles = params[3:]
    
    if not Vol_calc(unit_params, unit_angles, v_max):
        return 1e3
    
    # Calculate theoretical pattern
    alltheta2 = np.arange(min(theta2_exp)-1, max(theta2_exp)+1, peak_tol)
    _, _, theta2, _ = pyXRDCodes.generate_hkls(lamb, 10, unit_params, unit_angles)
    thetas2 = 2*np.array(theta2)
    if len(theta2) == 0:
        return 1e3

    mask = (thetas2 >= min(alltheta2)) & (thetas2 <= max(alltheta2))
    thetas2 = thetas2[mask]

    def make_binary_vector(peaks_theta):
        vec = np.zeros(len(alltheta2))
        for t in peaks_theta:
            idx = np.argmin(np.abs(alltheta2 - t))
            vec[idx] = 1
        return vec
    
    v = make_binary_vector(theta2_exp)
    u = make_binary_vector(thetas2)
    
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    if norm_u == 0 or norm_v == 0:
        return 1e3
    
    cosine_sim = dot_product / (norm_u * norm_v)
    fitness = (cosine_sim)**power
    
    return -fitness

def PSO(compute_fitness, observed_theta, early_stag=5, num_particles=30, max_iter=100, lamb=1.5406):
    # Define bounds for each parameter
    length_bounds = [1, 25]  # for a, b, c
    angle_bounds = [50, 125]  # for alpha, beta, gamma
    
    # Initialize particles with all 6 parameters
    particles = np.zeros((num_particles, 6))
    for i in range(num_particles):
        particles[i, :3] = np.random.uniform(length_bounds[0], length_bounds[1], 3)
        particles[i, 3:] = np.random.uniform(angle_bounds[0], angle_bounds[1], 3)
    
    velocities = np.zeros_like(particles)
    
    # Initialize best positions
    personal_best_positions = particles.copy()
    personal_best_scores = np.array([compute_fitness(pos, observed_theta, lamb) for pos in particles])
    
    global_best_idx = np.argmin(personal_best_scores)
    global_best_position = personal_best_positions[global_best_idx]
    global_best_score = personal_best_scores[global_best_idx]
    no_improvement = np.zeros(num_particles)
    
    # PSO parameters
    w = 0.7  # inertia weight
    c1 = 1.5  # cognitive component
    c2 = 1.5  # social component
    repulsion_factor = -0.3
    
    stagnated_particles = []
    stagnated_iter = []
    
    for iter_num in range(max_iter):
        iter_scores = []
        
        for i in range(num_particles):
            # Update velocity
            r1, r2 = np.random.rand(6), np.random.rand(6)
            v_hill = np.zeros(6)
            
            if len(stagnated_particles) != 0:
                for k in range(len(stagnated_particles)):
                    v_hill += (particles[i] - stagnated_particles[k])**2
            
            velocities[i] = (
                w * velocities[i] +
                c1 * r1 * (personal_best_positions[i] - particles[i]) +
                c2 * r2 * (global_best_position - particles[i]) +
                repulsion_factor * v_hill
            )
            
            # Update position with bounds checking
            particles[i] += velocities[i]
            
            # Apply bounds
            particles[i, :3] = np.clip(particles[i, :3], length_bounds[0], length_bounds[1])
            particles[i, 3:] = np.clip(particles[i, 3:], angle_bounds[0], angle_bounds[1])
            
            # Evaluate fitness
            current_score = compute_fitness(particles[i], observed_theta, lamb)
            iter_scores.append(current_score)
            
            # Update personal best
            if current_score < personal_best_scores[i]:
                personal_best_scores[i] = current_score
                personal_best_positions[i] = particles[i].copy()
                
                # Update global best
                if current_score < global_best_score:
                    global_best_score = current_score
                    global_best_position = particles[i].copy()
                    no_improvement[i] = 0
            else:
                no_improvement[i] += 1
            
            # Stagnation handling
            if no_improvement[i] >= early_stag:
                stagnated_particles.append(particles[i].copy())
                stagnated_iter.append(iter_num)
                
                # Random perturbation for each parameter
                perturbation = np.array([
                    np.random.choice([0.2, 0.5, 0.7, 1.2, 1.5, 1.7]) + np.random.uniform(-0.05, 0.05)
                    for _ in range(6)
                ])
                particles[i] *= perturbation
                
                # Reapply bounds after perturbation
                particles[i, :3] = np.clip(particles[i, :3], length_bounds[0], length_bounds[1])
                particles[i, 3:] = np.clip(particles[i, 3:], angle_bounds[0], angle_bounds[1])
                
                no_improvement[i] = 0
        
        print(f"Iteration {iter_num+1}, Best Fitness: {global_best_score:.4f}")
        print(f"Best params: a={global_best_position[0]:.2f}, b={global_best_position[1]:.2f}, c={global_best_position[2]:.2f}")
        print(f"Angles: α={global_best_position[3]:.1f}°, β={global_best_position[4]:.1f}°, γ={global_best_position[5]:.1f}°")
    
    return global_best_position, global_best_score, stagnated_particles, stagnated_iter

# Example usage
lamb = 1.5406
target_Atoms = ['Ti','Al']
target_unitparams = np.array([5, 5, 5, 90, 90, 90])
target_Positions = [
    [pyXRDCodes.abcToxyz([1/2, 1/3, 1/3], target_unitparams[:3], target_unitparams[3:])],
    [pyXRDCodes.abcToxyz([0, 0, 0], target_unitparams[:3], target_unitparams[3:]),
     pyXRDCodes.abcToxyz([1/2, 1/2, 1/2], target_unitparams[:3], target_unitparams[3:])]
]

# Generate expected peaks
data = pyXRDCodes.simulate_xrd(lamb, 10, target_unitparams[:3], target_unitparams[3:], target_Positions, target_Atoms)
observed_theta = np.round(np.array(data['two_thetas']), 2)

# Call PSO to optimize parameters
best_position, best_fitness, _, _ = PSO(compute_fitness, observed_theta, max_iter=40, num_particles=90, early_stag=5)
print("\nFinal Results:")
print(f"Best Parameters: a={best_position[0]:.2f}, b={best_position[1]:.2f}, c={best_position[2]:.2f}")
print(f"Angles: α={best_position[3]:.1f}°, β={best_position[4]:.1f}°, γ={best_position[5]:.1f}°")
print(f"Best Fitness: {best_fitness:.4f}")

Iteration 1, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 2, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 3, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 4, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 5, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 6, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 7, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 8, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 9, Best Fitness: -0.0637
Best params: a=19.46, b=9.09, c=18.24
Angles: α=116.4°, β=50.8°, γ=70.3°
Iteration 10, Best Fitness: 

---

Fixated to cubic

In [7]:
def Vol_calc(unit_params,unit_angles,V_max):
    a = pyXRDCodes.abcToxyz([1,0,0],unit_params,unit_angles)
    b = pyXRDCodes.abcToxyz([0,1,0],unit_params,unit_angles)
    c = pyXRDCodes.abcToxyz([0,0,1],unit_params,unit_angles)
    V = abs(np.dot(a,np.cross(b,c)))
    
    if V>=V_max:
        return False
    return True


def compute_fitness(unit_param,unit_angles, theta2_exp, lamb,power=1, peak_tol=0.02,v_max=3000):
    """
    Cosine similarity-based fitness with power scaling for peak matching
    
    Args:
        struct: Candidate structure
        theta2_exp: Experimental 2θ angles (degrees)
        int_exp: Experimental intensities
        lamb: Wavelength (Å)
        alltheta2: All possible theoretical 2θ angles
        power: Exponent for cosine similarity (higher = stricter)
        peak_tol: Peak matching tolerance (degrees)
        
    Returns:
        Fitness score (higher is better)
    """
    if not(Vol_calc(unit_param,unit_angles,v_max)):
        return 1e3
    # Calculate theoretical pattern
    alltheta2 = np.arange(min(theta2_exp)-1 , max(theta2_exp)+1,peak_tol)
    _, _, theta2, _ = pyXRDCodes.generate_hkls(lamb, 10, unit_param, unit_angles)
    thetas2 = 2*np.array(theta2)
    if len(theta2) == 0:
        return 1e3

    mask = (thetas2>=min(alltheta2))&(thetas2<=max(alltheta2))
    thetas2 = thetas2[mask]

    # Create binary peak vectors (1=peak, 0=no peak)
    def make_binary_vector(peaks_theta):
        vec = np.zeros(len(alltheta2))
        for t in peaks_theta:
            idx = np.argmin(np.abs(alltheta2 - t))
            vec[idx] = 1
        return vec
    
    # Experimental binary vector (only peaks above threshold)
    v = make_binary_vector(theta2_exp)
    u = make_binary_vector(thetas2)
    
    # Calculate cosine similarity
    dot_product = np.dot(u,v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    
    # Handle edge cases
    if norm_u == 0 or norm_v == 0:
        return 1e3
    
    cosine_sim = dot_product / (norm_u * norm_v)
    
    # Apply power scaling
    fitness = (cosine_sim)**power
    
    return -fitness #- atom_penalty  # Higher is better


def PSO(compute_fitness, observed_theta, early_stag=5, num_particles=30, max_iter=100, lamb=1.5406, order=10):
    # Define bounds
    param_bounds = [1,20]

    
    # Initialize population using Monte Carlo
    particles = np.random.choice(np.arange(param_bounds[0],param_bounds[1],0.1),num_particles)
    velocities = np.zeros_like(particles)
    
    # Initialize best positions
    personal_best_positions = particles.copy()
    personal_best_scores = np.array([
        compute_fitness( np.array([pos,pos,pos]), np.array([90,90,90]), observed_theta, lamb) 
        for pos in particles
    ])    

    global_best_idx = np.argmin(personal_best_scores)
    global_best_position = personal_best_positions[global_best_idx]
    global_best_score = personal_best_scores[global_best_idx]
    no_improvement = np.zeros(len(particles))
    
    # PSO parameters
    w = 1  # inertia weight
    c1 = 1.5  # cognitive component
    c2 = 0.5  # social component
    repulsion_factor = -0.3  # Strength of repulsion
        
    
    #====Define array with minimums
    #global_worst_idx = np.argmin(personal_best_scores)
    stagnated_particles = []
    stagnated_iter = []
    
    for iter_num in range(max_iter):
        iter_scores = []
 
        
        #=================STANDARD PSO UPDATE=========================
        for i in range(num_particles):
            # Update velocity
            r1, r2 = np.random.rand(), np.random.rand()
            v_hill = 0
            if len(stagnated_particles)!=0:
                for k in range(len(stagnated_particles)):
                    v_hill +=  (particles[i] - stagnated_particles[k][0])**2
            velocities[i] = (
                w * velocities[i] + 
                c1 * r1 * (personal_best_positions[i] - particles[i]) + #Own best
                c2 * r2 * (global_best_position - particles[i]) +#Global best
                repulsion_factor*v_hill
                #add new term of stagnation
            )
            
            # Update position with bounds checking
            particles[i] = particles[i] + velocities[i]
            if particles[i] < param_bounds[0]:
                particles[i] = param_bounds[0]
                velocities[i] *= -0.5  # Bounce back with damping
            if particles[i] > param_bounds[1]:
                particles[i] = param_bounds[1]
                velocities[i] *= -0.5

            
            # Evaluate fitness
            current_score = compute_fitness(np.array([particles[i],particles[i],particles[i]]), np.array([90,90,90])
                                                      , observed_theta, lamb)
            iter_scores.append(current_score)

            # Update personal best
            if current_score < personal_best_scores[i]:
                personal_best_scores[i] = current_score
                personal_best_positions[i] = particles[i].copy()
                
                # Update global best
                if current_score < global_best_score:
                    global_best_score = current_score
                    global_best_position = particles[i].copy()
                    no_improvement[i] = 0  # Reset stagnation counter
            else:
                no_improvement[i]+=1            
            #=================STAGNATION HANDLING WITH REPULSION=========================
            if no_improvement[i] >= early_stag:            
                stagnated_particles.append([particles[i]])
                stagnated_iter.append(iter_num)
                particles[i] = particles[i]*(np.random.choice([0.2,0.5,0.7,1.2,1.5,1.7])+np.random.uniform(-0.05,0.05))
                if particles[i] < param_bounds[0]:
                    particles[i] = param_bounds[0]
                    velocities[i] *= -0.5  # Start going towards where it was
                if particles[i] > param_bounds[1]:
                    particles[i] = param_bounds[1]
                    velocities[i] *= -0.5 # Start going towards where it was
                
                no_improvement[i] = 0
            #==========================================
        
        print(f"Iteration {iter_num+1}, Best Fitness: {global_best_score:.4f}, Stagnated: {len(stagnated_particles)}")
        print(f"best param {global_best_position}")
    
    return global_best_position, global_best_score,stagnated_particles,stagnated_iter

# Your target structure (Ti-C-Al-O)
lamb = 1.5406
target_Atoms = ['Ti','Al']
target_unitparams = np.array([5, 5, 5, 90, 90, 90])
target_Positions = [
        [pyXRDCodes.abcToxyz([1/2, 1/3, 1/3], target_unitparams[:3], target_unitparams[3:])],
        [pyXRDCodes.abcToxyz([0, 0, 0], target_unitparams[:3], target_unitparams[3:]),
         pyXRDCodes.abcToxyz([1/2, 1/2, 1/2], target_unitparams[:3], target_unitparams[3:])]
    ]

# Generate expected peaks
data = pyXRDCodes.simulate_xrd(lamb, 10, target_unitparams[:3], target_unitparams[3:], target_Positions, target_Atoms)
observed_theta = np.round(np.array(data['two_thetas']), 2)
U = data['intensities']
observed_int = 0

print('BEST' , compute_fitness(target_unitparams[:3], target_unitparams[3:], observed_theta,lamb))
# Call PSO to optimize parameters
best_position, best_fitness,_,_ = PSO(compute_fitness, observed_theta,max_iter=40,num_particles = 90,early_stag = 5)
print(f"Best Parameters: {best_position}, Fitness: {best_fitness}")

BEST -0.6666666666666666
Iteration 1, Best Fitness: -0.1279, Stagnated: 0
best param 3.872966524873626
Iteration 2, Best Fitness: -0.1279, Stagnated: 0
best param 3.872966524873626
Iteration 3, Best Fitness: -0.1279, Stagnated: 0
best param 3.872966524873626
Iteration 4, Best Fitness: -0.1279, Stagnated: 0
best param 3.872966524873626
Iteration 5, Best Fitness: -0.1279, Stagnated: 45
best param 3.872966524873626
Iteration 6, Best Fitness: -0.1279, Stagnated: 77
best param 3.872966524873626
Iteration 7, Best Fitness: -0.1279, Stagnated: 89
best param 3.872966524873626
Iteration 8, Best Fitness: -0.1279, Stagnated: 90
best param 3.872966524873626
Iteration 9, Best Fitness: -0.1279, Stagnated: 90
best param 3.872966524873626
Iteration 10, Best Fitness: -0.1279, Stagnated: 135
best param 3.872966524873626
Iteration 11, Best Fitness: -0.1279, Stagnated: 167
best param 3.872966524873626
Iteration 12, Best Fitness: -0.1279, Stagnated: 179
best param 3.872966524873626
Iteration 13, Best Fitnes

HExagonal

In [8]:
def PSO(compute_fitness, observed_theta, early_stag=5, num_particles=30, max_iter=100, lamb=1.5406):
    # Define bounds for hexagonal parameters
    a_bounds = [1, 20]  # bounds for a (and b since a = b)
    c_bounds = [1, 20]  # bounds for c (independent of a/b)
    
    # Initialize particles - each particle has [a, c] (since a=b and angles are fixed)
    particles = np.zeros((num_particles, 2))
    particles[:, 0] = np.random.uniform(a_bounds[0], a_bounds[1], num_particles)  # a
    particles[:, 1] = np.random.uniform(c_bounds[0], c_bounds[1], num_particles)  # c
    
    velocities = np.zeros_like(particles)
    
    # Initialize best positions
    personal_best_positions = particles.copy()
    personal_best_scores = np.array([
        compute_fitness(
            np.array([pos[0], pos[0], pos[1]]),  # a, b=a, c
            np.array([90, 90, 120]),  # fixed angles for hexagonal
            observed_theta, 
            lamb
        ) 
        for pos in particles
    ])    

    global_best_idx = np.argmin(personal_best_scores)
    global_best_position = personal_best_positions[global_best_idx]
    global_best_score = personal_best_scores[global_best_idx]
    no_improvement = np.zeros(num_particles)
    
    # PSO parameters
    w = 1  # inertia weight
    c1 = 1.5  # cognitive component
    c2 = 0.5  # social component
    repulsion_factor = -0.3  # Strength of repulsion
        
    stagnated_particles = []
    stagnated_iter = []
    
    for iter_num in range(max_iter):
        iter_scores = []
        
        for i in range(num_particles):
            # Update velocity
            r1, r2 = np.random.rand(2), np.random.rand(2)
            v_hill = np.zeros(2)
            if len(stagnated_particles) != 0:
                for k in range(len(stagnated_particles)):
                    v_hill += (particles[i] - stagnated_particles[k])**2
            
            velocities[i] = (
                w * velocities[i] + 
                c1 * r1 * (personal_best_positions[i] - particles[i]) +  # Own best
                c2 * r2 * (global_best_position - particles[i]) +  # Global best
                repulsion_factor * v_hill
            )
            
            # Update position with bounds checking
            particles[i] += velocities[i]
            
            # Apply bounds for a (and b)
            particles[i, 0] = np.clip(particles[i, 0], a_bounds[0], a_bounds[1])
            # Apply bounds for c
            particles[i, 1] = np.clip(particles[i, 1], c_bounds[0], c_bounds[1])

            # Evaluate fitness with hexagonal constraints
            current_score = compute_fitness(
                np.array([particles[i, 0], particles[i, 0], particles[i, 1]]),  # a, b=a, c
                np.array([90, 90, 120]),  # fixed hexagonal angles
                observed_theta, 
                lamb
            )
            iter_scores.append(current_score)

            # Update personal best
            if current_score < personal_best_scores[i]:
                personal_best_scores[i] = current_score
                personal_best_positions[i] = particles[i].copy()
                
                # Update global best
                if current_score < global_best_score:
                    global_best_score = current_score
                    global_best_position = particles[i].copy()
                    no_improvement[i] = 0  # Reset stagnation counter
            else:
                no_improvement[i] += 1            
            
            # Stagnation handling with repulsion
            if no_improvement[i] >= early_stag:            
                stagnated_particles.append(particles[i].copy())
                stagnated_iter.append(iter_num)
                
                # Apply random perturbation to both a and c
                perturbation = np.array([
                    np.random.choice([0.2, 0.5, 0.7, 1.2, 1.5, 1.7]) + np.random.uniform(-0.05, 0.05),
                    np.random.choice([0.2, 0.5, 0.7, 1.2, 1.5, 1.7]) + np.random.uniform(-0.05, 0.05)
                ])
                particles[i] *= perturbation
                
                # Reapply bounds after perturbation
                particles[i, 0] = np.clip(particles[i, 0], a_bounds[0], a_bounds[1])
                particles[i, 1] = np.clip(particles[i, 1], c_bounds[0], c_bounds[1])
                
                no_improvement[i] = 0
        
        print(f"Iteration {iter_num+1}, Best Fitness: {global_best_score:.4f}, Stagnated: {len(stagnated_particles)}")
        print(f"Best params: a=b={global_best_position[0]:.2f} Å, c={global_best_position[1]:.2f} Å")
    
    # Return the full unit cell parameters (a, b=a, c, 90, 90, 120)
    final_params = np.array([
        global_best_position[0],  # a
        global_best_position[0],  # b = a
        global_best_position[1],  # c
        90, 90, 120  # fixed angles
    ])
    
    return final_params, global_best_score, stagnated_particles, stagnated_iter

# Example usage (same as before)
lamb = 1.5406
target_Atoms = ['Ti','Al']
target_unitparams = np.array([3, 3, 20, 90, 90, 120])  # Note: This should be hexagonal for realistic test
target_Positions = [
    [pyXRDCodes.abcToxyz([1/2, 1/3, 1/3], target_unitparams[:3], target_unitparams[3:])],
    [pyXRDCodes.abcToxyz([0, 0, 0], target_unitparams[:3], target_unitparams[3:]),
     pyXRDCodes.abcToxyz([1/2, 1/2, 1/2], target_unitparams[:3], target_unitparams[3:])]
]

# Generate expected peaks
data = pyXRDCodes.simulate_xrd(lamb, 10, target_unitparams[:3], target_unitparams[3:], target_Positions, target_Atoms)
observed_theta = np.round(np.array(data['two_thetas']), 2)

print('BEST', compute_fitness(target_unitparams[:3], target_unitparams[3:], observed_theta, lamb))

# Call PSO to optimize parameters
best_position, best_fitness, _, _ = PSO(compute_fitness, observed_theta, max_iter=40, num_particles=90, early_stag=5)
print(f"\nBest Parameters: a=b={best_position[0]:.2f} Å, c={best_position[2]:.2f} Å")
print(f"Angles: α=β=90°, γ=120°")
print(f"Best Fitness: {best_fitness:.4f}")

BEST -0.537653269544012
Iteration 1, Best Fitness: -0.0513, Stagnated: 0
Best params: a=b=18.13 Å, c=1.12 Å
Iteration 2, Best Fitness: -0.0513, Stagnated: 0
Best params: a=b=18.13 Å, c=1.12 Å
Iteration 3, Best Fitness: -0.0844, Stagnated: 0
Best params: a=b=20.00 Å, c=1.07 Å
Iteration 4, Best Fitness: -0.1088, Stagnated: 0
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 5, Best Fitness: -0.1088, Stagnated: 19
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 6, Best Fitness: -0.1088, Stagnated: 51
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 7, Best Fitness: -0.1088, Stagnated: 75
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 8, Best Fitness: -0.1088, Stagnated: 87
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 9, Best Fitness: -0.1088, Stagnated: 90
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 10, Best Fitness: -0.1088, Stagnated: 109
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 11, Best Fitness: -0.1088, Stagnated: 141
Best params: a=b=5.09 Å, c=20.00 Å
Iteration 12, Best Fitness: -0.1088, Stag